In [1]:
import torch
from torch import nn
import matplotlib.pyplot as plt

torch.__version__

'2.0.1+cu117'

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [3]:
weight = 0.7
bias = 0.3

start = 0
end = 10000000
step = 0.02

X = torch.arange(start=start, end=end, step=step).unsqueeze(dim=1)
y = weight * X + bias

X[:10], y[:10]

(tensor([[0.0000],
         [0.0200],
         [0.0400],
         [0.0600],
         [0.0800],
         [0.1000],
         [0.1200],
         [0.1400],
         [0.1600],
         [0.1800]]),
 tensor([[0.3000],
         [0.3140],
         [0.3280],
         [0.3420],
         [0.3560],
         [0.3700],
         [0.3840],
         [0.3980],
         [0.4120],
         [0.4260]]))

In [4]:
train_split = int(0.8 * len(X))
X_train = X[:train_split]
y_train = y[:train_split]

X_test = X[train_split:]
y_test = y[train_split:]

len(X_train), len(y_train), len(X_test), len(y_test)

(400000000, 400000000, 100000000, 100000000)

In [5]:
def plot_predictions(train_data=X_train, 
                     train_labels=y_train, 
                     test_data=X_test, 
                     test_labels=y_test, 
                     predictions=None):
    plt.figure(figsize=(5, 3.5))
    plt.scatter(train_data, train_labels, c="b", s=4, label="Dados de treinamento")
    plt.scatter(test_data, test_labels, c="g", s=4, label="Dados de teste")
    if predictions is not None:
        # Plot the predictions in red (predictions were made on the test data)
        plt.scatter(test_data, predictions, c="r", s=4, label="Predictions")


    plt.legend(prop={"size":14})

In [ ]:
plot_predictions(X_train, y_train, X_test, y_test)

In [6]:
class LinearRegressionModelV2(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear_layer = nn.Linear(in_features=1, out_features=1)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.linear_layer(x)
    
torch.manual_seed(42)
model_1 = LinearRegressionModelV2()
model_1, model_1.state_dict()

(LinearRegressionModelV2(
   (linear_layer): Linear(in_features=1, out_features=1, bias=True)
 ),
 OrderedDict([('linear_layer.weight', tensor([[0.7645]])),
              ('linear_layer.bias', tensor([0.8300]))]))

In [7]:
model_1.to(device=device)
next(model_1.parameters()).device

device(type='cuda', index=0)

In [14]:
# criar loss function
loss_fn = nn.L1Loss()
print(next(model_1.parameters()))
optimizer = torch.optim.AdamW(params=model_1.parameters(), lr=0.01)

<generator object Module.parameters at 0x0000020EDD379380>


In [13]:
torch.manual_seed(42)
epochs = 1000000

X_train = X_train.to(device)
X_test = X_test.to(device)
y_train = y_train.to(device)
y_test = y_test.to(device)

for epoch in range(epochs):
    model_1.train()

    y_pred = model_1(X_train) #forward pass
    loss = loss_fn(y_pred, y_train) #calcula loss
    optimizer.zero_grad() #zero grad optimizer
    loss.backward()
    optimizer.step()

    # Testando
    with torch.inference_mode():
        test_pred = model_1(X_test)
        test_loss = loss_fn(test_pred, y_test)

    if epoch % 100 == 0:
        print(f"Epoch: {epoch} | Train loss: {loss} | Test loss: {test_loss}")

Epoch: 0 | Train loss: 247332.84375 | Test loss: 465812.65625
Epoch: 100 | Train loss: 8356.1376953125 | Test loss: 1516.2684326171875
Epoch: 200 | Train loss: 10021.6455078125 | Test loss: 11391.5947265625
Epoch: 300 | Train loss: 4124.20947265625 | Test loss: 5551.92236328125
Epoch: 400 | Train loss: 2930.20947265625 | Test loss: 8561.8623046875


KeyboardInterrupt: 

In [ ]:
print("The model learned the following values for weights and bias:")
print(model_1.state_dict())
print("\nAnd the original values for weights and bias are:")
print(f"weights: {weight}, bias: {bias}")

In [ ]:
# Realizando predições
model_1.eval() # Coloca o modelo em modo de avaliação
input_to_predict = torch.arange(start=10, end=11, dtype=torch.float32).unsqueeze(dim=1)
print(input_to_predict)
input_to_predict = input_to_predict.to(device)
with torch.inference_mode():
    y_preds = model_1(input_to_predict)

y_preds

In [ ]:
plot_predictions(predictions=y_preds.cpu())

# Salvar e carregar modelos